In [2]:
import os
os.environ["KACHERY_CLOUD_DIR"] = "/home/labuser/.kachery-cloud"
os.environ["KACHERY_ZONE"] = "default"


In [3]:
%run common_imports.py

%matplotlib qt
%config InlineBackend.figure_format = 'retina'

%reload_ext autoreload
%autoreload 2

dj.config['display.limit'] = 10**3  
plt.style.use("seaborn-v0_8-talk")

os.environ["SPYGLASS_USE_TRANSACTIONS"] = "1" 
os.environ['KACHERY_API_KEY'] = "RhysjLwgmBAt2ObCyXXaDnqAv2kTdYRa"

import sys
import notebooks.sorting_multiprocessing as smp
import spyglass.spikesorting.v1 as sgs
import spikeinterface as si
import spikeinterface.widgets as sw
import spikeinterface.core as sc
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
from spyglass.spikesorting.analysis.v1.group import SortedSpikesGroup


[2025-12-26 11:01:29,386][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-26 11:01:29,400][INFO]: Connected anirudh@172.16.102.154:3306


### Set Sort Group

In [10]:
# set sort group, using reference electrodes from the corpus callosum

references = {}
hpc_reference = "0"
for index, rows in pd.DataFrame((sgc.ElectrodeGroup() & session_restrict).fetch()).iterrows():
    e_group = rows["electrode_group_name"]
    region = rows["region_id"]
    if (region != 11):
        references.update({e_group:hpc_reference})
    else:
        references.update({e_group:-1}) # no reference for mpfc, ofc
        

sgs.SortGroup.set_group_by_shank(nwb_file_name=nwb_copy_file_name, references=references)

In [11]:
sgs.SortGroup() & session_restrict

nwb_file_name name of the NWB file,sort_group_id,sort_reference_electrode_id the electrode to use for referencing
wilbur20210512_.nwb,0,0
wilbur20210512_.nwb,1,0
wilbur20210512_.nwb,2,0
wilbur20210512_.nwb,3,0
wilbur20210512_.nwb,4,0
wilbur20210512_.nwb,5,0
wilbur20210512_.nwb,6,0
wilbur20210512_.nwb,7,0
wilbur20210512_.nwb,8,0
wilbur20210512_.nwb,9,0


### SpikeSortingRecording

In [3]:
sgs.SpikeSortingPreprocessingParameters()

preproc_param_name,preproc_params
default,=BLOB=
Hippocampups_default,=BLOB=


In [4]:
from spyglass.common.common_interval import IntervalList

base_key = {"nwb_file_name": nwb_copy_file_name}
epoch_names = ["02_wtrack", "04_wtrack", "06_wtrack", "08_wtrack"]

valid_times_list = []
for name in epoch_names:
    vt = (IntervalList & {**base_key, "interval_list_name": name}).fetch1("valid_times")
    valid_times_list.append(vt)

import numpy as np
all_valid_times = np.vstack(valid_times_list)

IntervalList.insert1(
    {
        **base_key,
        "interval_list_name": "wtrack_all",
        "valid_times": all_valid_times,
    },
    skip_duplicates=True,
)


In [5]:
from itertools import product
sort_group_ids = np.arange(24, 36) # using only PFC data
# interval_lists = ["02_wtrack","04_wtrack", "06_wtrack", "08_wtrack"]
interval_lists = ["wtrack_all"]

for sort_group_id, interval_list in product(sort_group_ids, interval_lists): 
    key = {
        "nwb_file_name": nwb_copy_file_name,
        "sort_group_id": sort_group_id,
        "preproc_param_name": "default",
        "interval_list_name": interval_list,
        "team_name": "NeuroACT",
    }
    sgs.SpikeSortingRecordingSelection.insert_selection(key)

[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:22][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:23][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:23][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:23][WARNING] Spyglass: Similar row(s) already inserted.
[22:20:23][WARNING] Spyglass: Similar row(s) already inserted.


In [12]:
# default parameters: bandpass filter (600 - 6000 Hz), apply whitening
module_dir = os.getcwd()         
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)
keys = ((sgs.SpikeSortingRecordingSelection() & session_restrict & {"interval_list_name": "wtrack_all"}) - sgs.SpikeSortingRecording()).fetch("KEY")
pfc_keys = keys.copy()
procs = smp.launch_workers((sgs.SpikeSortingRecording & session_restrict), keys, n_workers=8) # run 8 parallel processes
for p in procs: p.join()


[2025-12-19 22:22:09,537][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,538][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,544][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,549][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,551][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,552][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,552][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,557][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,560][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,562][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,562][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,563][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,565][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 22:22:09,576][INFO]: Connected anirudh@172.16.102.154:330

### Artifact Detection

In [13]:
recording_ids = pfc_keys.copy()
for recording_id in recording_ids:
   (sgs.ArtifactDetectionSelection).insert_selection({"recording_id": recording_id["recording_id"], # skip_duplicates = True by default
                                "artifact_param_name": "default"})


[2025-12-19 23:39:21,539][WARNING]: MySQL server has gone away. Reconnecting to the server.


In [14]:
keys = (sgs.ArtifactDetectionSelection() & pfc_keys).fetch("KEY") 
procs = smp.launch_workers(sgs.ArtifactDetection, keys, n_workers=12)
for p in procs: p.join()

[2025-12-19 23:39:22,884][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,884][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,885][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,887][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,888][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,892][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,895][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,898][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,901][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,901][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,903][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,904][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,907][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-19 23:39:22,911][INFO]: Connected anirudh@172.16.102.154:3306

In [15]:
sgs.ArtifactDetection() & session_restrict

[2025-12-20 00:30:29,866][WARNING]: MySQL server has gone away. Reconnecting to the server.


artifact_id
09c4ffc8-19f3-4025-829a-660d8c4fcf2d
0fa66081-d12a-4580-ab58-a4aa9038f048
171a433f-7442-45fb-b17d-3c80ad8bc1e3
1c4f8899-3407-4647-ae97-ef9e5cb70f36
1ea1ecae-cb1e-4bf8-946f-6254ad8598f6
2567db23-6c19-4522-9336-7778e43279fa
26d767e9-28c0-42ee-a2ae-1493cd595a74
2878b55f-7024-45c8-bb52-9b7e69d1dcf9
3b508a82-312a-44d7-911a-2d04c073b42c
3bd8cfa9-60b1-4436-8c0d-7b5adc765fea


### SpikeSorting

In [16]:
(sgs.SpikeSorterParameters() &  {"sorter_param_name": "pfc_default"}).fetch1()

{'sorter': 'mountainsort4',
 'sorter_param_name': 'pfc_default',
 'sorter_params': {'detect_sign': -1,
  'adjacency_radius': 115,
  'filter': False,
  'whiten': True,
  'num_workers': 4,
  'clip_size': 30,
  'detect_threshold': 3,
  'detect_interval': 10,
  'freq_min': 300,
  'freq_max': 6000}}

In [17]:
recording_ids = (sgs.SpikeSortingRecordingSelection() & session_restrict & pfc_keys).fetch("recording_id")
sorter = "mountainsort4"
for recording_id in recording_ids:
    common_key = {
        "recording_id": recording_id,
        "sorter": sorter,
        "nwb_file_name": nwb_copy_file_name,
        "interval_list_name": str(
            (
                sgs.ArtifactDetectionSelection 
                & {"recording_id": recording_id}
            ).fetch1("artifact_id")
        ),
    } ## sgc.IntervalList contains intervals named by detected artifacts


    key = {
    **common_key,
    "sorter_param_name": "pfc_default",
}

    sgs.SpikeSortingSelection.insert_selection(key)

In [18]:
keys = (sgs.SpikeSortingSelection() & pfc_keys).fetch("KEY")
procs = smp.launch_workers((sgs.SpikeSorting), keys, n_workers=4)
for p in procs: p.join()

[2025-12-20 00:30:31,245][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,259][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,264][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,269][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,273][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,278][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,284][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:31,286][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,651][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,653][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,663][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,669][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,683][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 00:30:32,684][INFO]: Connecting anirudh@172.16.102.154:330

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[01:20:30][INFO] Spyglass: Writing new NWB file wilbur20210512_RYHBPYGTPY.nwb
[01:20:31][INFO] Spyglass: Writing new NWB file wilbur20210512_CFFZ30CF9X.nwb
[01:20:42][INFO] Spyglass: Writing new NWB file wilbur20210512_FVFXKC5SDK.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_RYHBPYGTPY.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuse

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[01:58:32][INFO] Spyglass: Writing new NWB file wilbur20210512_JC4ETKU8EJ.nwb
[01:59:02][INFO] Spyglass: Writing new NWB file wilbur20210512_BQGX95E88A.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_JC4ETKU8EJ.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmp_txgbwur'>
  _warnings.warn(warn_message, R

...

[02:01:33][INFO] Spyglass: Writing new NWB file wilbur20210512_AETNUQJ34M.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_AETNUQJ34M.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmpatujniox'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting:   0%|          | 0/1 [00:00<?, ?it/s][2025-12-20

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[02:03:46][INFO] Spyglass: Writing new NWB file wilbur20210512_L8VIX2VHQO.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_L8VIX2VHQO.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmpedrg9_ek'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting:   0%|          | 0/1 [00:00<?, ?it/s][2025-12-20

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[02:29:43][INFO] Spyglass: Writing new NWB file wilbur20210512_66U7M454FA.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_66U7M454FA.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmpkxam7vr2'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting: 100%|██████████| 1/1 [30:46<00:00, 1846.71s/it]


In [19]:
sgs.SpikeSorting()

[2025-12-20 02:41:11,428][WARNING]: MySQL server has gone away. Reconnecting to the server.


sorting_id,analysis_file_name name of the file,object_id Object ID for the sorting in NWB file,"time_of_sort in Unix time, to the nearest second"
014c4043-4900-469b-85c3-4e2e96c9eb3e,J1620210722_TWG1PXM73Q.nwb,6b3d39e8-ec1d-4706-8d1a-b0db5173cce3,1762939225
039cacd1-def5-41c1-ba5d-12fbccc65033,J1620210722_MEPU6GNSIS.nwb,8d34636f-e34f-4c99-ba33-66a5ee2cd750,1762939509
04abb6b4-7cf8-408f-bce0-97baa216e6bd,wilbur20210512_VEL9SBH0M6.nwb,e47ee9f4-313f-4d40-90ce-160f17622985,1765559048
05274957-75da-4792-9b81-a045a5deec72,wilbur20210512_5LTKCZWWDL.nwb,5268d301-a794-43ef-b304-ceb67ba38b43,1765558606
0600316d-8b68-4411-a674-0d6f71c6f50e,wilbur20210512_CJHGUYLO0U.nwb,b6c82c9e-9f9f-4c0b-b0a3-6099fb74cfa0,1765557058
0c0d6f10-5af6-4add-beac-67f101ecb017,wilbur20210512_CFFZ30CF9X.nwb,b08dc76d-f8fc-44db-a29a-029f2ccc2f8d,1766173817
0e42a00b-2af3-424c-a308-f2322ca6b2eb,wilbur20210512_ORRWO1VXG8.nwb,9c0049f3-7ab0-4ba1-b53f-71d3b728e7f0,1765556715
0e814145-3137-4e99-aff8-c59ce673069f,J1620210722_4935GQ78JX.nwb,9efb67a8-a7fc-4044-b430-d8b029617476,1762838701
103990e9-2063-4541-906e-5c69fdf42339,wilbur20210512_66U7M454FA.nwb,4fd41536-0fdc-422c-bb03-95542e1a443d,1766177969
11d09ce8-1630-4fd9-b8c9-889f17c7940b,wilbur20210512_SKD2Z545GI.nwb,a0a31af0-3d05-4c49-851a-76ae5abf58f3,1765557812


### Curation

#### Insert into CurationV1

In [20]:
sorting_ids = (sgs.SpikeSortingSelection & pfc_keys & session_restrict).fetch("sorting_id")
for sorting_id in sorting_ids:
    try:
        sgs.CurationV1.insert_curation(sorting_id = sorting_id)
    except KeyError:
        idx = sorting_id["sorting_id"]
        print(f"No units found in sorting {idx}!")


[02:41:32][INFO] Spyglass: Writing new NWB file wilbur20210512_FDK5EKGDLH.nwb
[02:42:34][INFO] Spyglass: Writing new NWB file wilbur20210512_S7W1M5S6FA.nwb
[02:43:38][INFO] Spyglass: Writing new NWB file wilbur20210512_UZGK0591Q8.nwb
[02:44:41][INFO] Spyglass: Writing new NWB file wilbur20210512_KWSSJB2PK4.nwb
[02:45:42][INFO] Spyglass: Writing new NWB file wilbur20210512_4DE8P5VUNN.nwb
[02:46:44][INFO] Spyglass: Writing new NWB file wilbur20210512_H6OC9EPRN3.nwb
[02:47:46][INFO] Spyglass: Writing new NWB file wilbur20210512_4OOT18NC44.nwb
[02:48:48][INFO] Spyglass: Writing new NWB file wilbur20210512_WFNGPHQWF3.nwb
[02:49:49][INFO] Spyglass: Writing new NWB file wilbur20210512_ZGFM692G5L.nwb
[02:50:51][INFO] Spyglass: Writing new NWB file wilbur20210512_8HRX3PVODJ.nwb
[02:51:53][INFO] Spyglass: Writing new NWB file wilbur20210512_GOF69J1QUX.nwb
[02:52:56][INFO] Spyglass: Writing new NWB file wilbur20210512_G1FFQTPAEJ.nwb


In [21]:
sgs.CurationV1() 

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
014c4043-4900-469b-85c3-4e2e96c9eb3e,0,-1,J1620210722_VLYRZRZ48U.nwb,335df2aa-ffac-410c-a7f8-2bf12668f6ce,0,
014c4043-4900-469b-85c3-4e2e96c9eb3e,1,0,J1620210722_HYIE1SJ4IH.nwb,529764f3-06b7-495c-a629-ba0dafda862a,0,after metric curation
014c4043-4900-469b-85c3-4e2e96c9eb3e,2,1,J1620210722_4N1M5XMYJW.nwb,1ab74bf2-11e7-49b5-986a-e939a434862d,0,
039cacd1-def5-41c1-ba5d-12fbccc65033,0,-1,J1620210722_WMJIHCKMWY.nwb,3e1de769-564a-4b02-bebe-ef36448f253e,0,
039cacd1-def5-41c1-ba5d-12fbccc65033,1,0,J1620210722_ZMBP6BE4IL.nwb,d8650c20-294a-40db-8d79-a0f83b0c825c,0,after metric curation
039cacd1-def5-41c1-ba5d-12fbccc65033,2,1,J1620210722_ZQUAH2BABW.nwb,0ce23c4d-b8bb-4da4-b1dc-752729d5569f,0,
04abb6b4-7cf8-408f-bce0-97baa216e6bd,0,-1,wilbur20210512_523HGUG95I.nwb,24d648bf-b44f-49a3-a6e5-0e9dfe6b557b,0,
04abb6b4-7cf8-408f-bce0-97baa216e6bd,1,0,wilbur20210512_TA2BLFJ11G.nwb,665adf72-4ebd-40c1-809c-a9b2ec04e239,0,after metric curation
04abb6b4-7cf8-408f-bce0-97baa216e6bd,2,1,wilbur20210512_I8KHMEKIAI.nwb,f1ff4ef5-3f5d-4901-b8f4-5dafe38df457,0,after FigURL curation
05274957-75da-4792-9b81-a045a5deec72,0,-1,wilbur20210512_HNEL79YCNG.nwb,916cc6d7-26d3-4297-8629-8ee7d976ee64,0,


#### Round 1: Automatic Curation

In [22]:
#insert parameters
label_params = {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']]}
merge_params = {}

sgs.MetricCurationParameters.insert1({"metric_curation_param_name": "noise_overlap_0.03",
                                      "label_params": label_params,
                                      "merge_params": merge_params
                                      }, skip_duplicates = True)

In [23]:
# insert into selection table
for sorting_idx in range(len((sgs.CurationV1() & session_restrict  & pfc_keys).fetch("sorting_id"))):
    key = {
        "sorting_id": (sgs.CurationV1() & session_restrict).fetch("sorting_id")[sorting_idx],
        "curation_id": 0,
        "waveform_param_name": "default_whitened",
        "metric_param_name": "franklab_default",
        "metric_curation_param_name": "noise_overlap_0.03",
    }
    sgs.MetricCurationSelection.insert_selection(key)
sgs.MetricCurationSelection()

[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38][WARNING] Spyglass: This row has already been inserted.
[02:53:38]

metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
02480e53-2868-4901-b26d-66e48f583d53,4f9c8ea6-bffb-476d-8908-a14f3fbe5aa7,0,default_whitened,franklab_default,noise_overlap_0.03
045708cb-a7e7-4f2e-8930-e376376dfb80,f3df9cb1-5729-4521-8624-1dd556b3adb2,0,default_whitened,franklab_default,noise_overlap_0.03
07b98aa8-2003-4f04-bab9-4c5085fa893a,430212fc-4265-480f-bed3-45387d7272d9,0,default_whitened,franklab_default,noise_overlap_0.03
0ec9b6d9-6f72-4121-9d55-9f8b90d24f04,7d115bb4-fdd3-4701-8369-e67eb32475ab,0,default_whitened,franklab_default,noise_overlap_0.03
117397f5-9d53-4923-b2d7-3651d116e931,589b0837-0196-41b2-984c-7f9855716898,0,default_whitened,franklab_default,noise_overlap_0.03
13e06c7c-e029-4f66-a06d-c71ae9505b05,fe96651e-adff-4bb3-9a23-0a2cb1eeab1c,0,default_whitened,franklab_default,noise_overlap_0.03
173402d5-01d7-473a-bb9f-6c2c8aaaf6e5,d83385c9-1e7c-480b-a3e0-58506057b775,0,default_whitened,franklab_default,noise_overlap_0.03
1e159fbe-72e7-46c6-9edf-6d41c38b6c13,b0482b2a-33b6-4964-b735-854c54cf6980,0,default_whitened,franklab_default,noise_overlap_0.03
1f8c96e4-bea2-4e57-aa5a-075fda265d83,f36656df-8942-4583-8a0e-f152dd36bc4e,2,default_whitened,franklab_default,pfc_round_2
2132d139-bb74-45e9-ab06-07e591b7be90,f3ca2fee-2b0a-4277-804f-f58b1dc0c3aa,0,default_whitened,franklab_default,noise_overlap_0.03


In [24]:
keys = (sgs.MetricCurationSelection() - sgs.MetricCuration()).fetch("KEY")
procs = smp.launch_workers((sgs.MetricCuration), keys, n_workers=4)
for p in procs: p.join()


[2025-12-20 02:53:42,087][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,090][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,095][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,100][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,101][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,104][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,112][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:42,114][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,495][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,497][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,507][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,511][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,511][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 02:53:43,514][INFO]: Connecting anirudh@172.16.102.154:330

In [25]:
sgs.MetricCuration()

[2025-12-20 03:45:59,527][WARNING]: MySQL server has gone away. Reconnecting to the server.


metric_curation_id,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
02480e53-2868-4901-b26d-66e48f583d53,J1620210722_TVYF18LDF6.nwb,ad72bebc-a0bb-4d54-b58a-0849e881223e
045708cb-a7e7-4f2e-8930-e376376dfb80,wilbur20210512_GU99WD3YK1.nwb,c27d8560-f44e-4e05-961c-2688051f8083
07b98aa8-2003-4f04-bab9-4c5085fa893a,J1620210722_A21KR53CUW.nwb,eb78fee5-dc8f-44d0-bd6b-52b32d93b3cf
0ec9b6d9-6f72-4121-9d55-9f8b90d24f04,J1620210722_VI5FPZC0Y6.nwb,24e7b6a6-cb5f-4d1a-b7d6-149139f4da0c
117397f5-9d53-4923-b2d7-3651d116e931,J1620210722_VEB5FXEJCG.nwb,7e80a9b0-d9a8-41f1-8f0b-10a0ce630bc8
13e06c7c-e029-4f66-a06d-c71ae9505b05,wilbur20210512_WGE8E2GXJH.nwb,94a780d1-b403-4d2c-97be-1c1d51ae5cce
173402d5-01d7-473a-bb9f-6c2c8aaaf6e5,J1620210722_N9N2QDLG9Z.nwb,73b0e071-5490-4060-8237-a669c268bd9e
1e159fbe-72e7-46c6-9edf-6d41c38b6c13,J1620210722_Z6LVJRF754.nwb,b2777ac5-d912-47c2-b306-ad8ea0f34d4b
1f8c96e4-bea2-4e57-aa5a-075fda265d83,J1620210722_MK40ZSE2EK.nwb,7edaf614-d0c2-4649-9207-f61fca95d6d1
2132d139-bb74-45e9-ab06-07e591b7be90,J1620210722_78MBASRWQO.nwb,de6abdc0-1a81-4d3a-851a-6255f41aaee0


#### Round 2: Manual curation (FigURL)

In [64]:
recording_ids = ((sgs.SpikeSortingRecordingSelection() & session_restrict & {"interval_list_name": "wtrack_all"})* sgs.SpikeSortingRecording()).fetch("KEY")
sorting_ids = (sgs.SpikeSortingSelection() & recording_ids).fetch("KEY")
metric_curation_ids = (sgs.MetricCurationSelection() & sorting_ids).fetch("KEY")

In [66]:
for metric_curation_id in metric_curation_ids:
    labels = sgs.MetricCuration.get_labels(metric_curation_id)
    merge_groups = sgs.MetricCuration.get_merge_groups(metric_curation_id)
    metrics = sgs.MetricCuration.get_metrics(metric_curation_id)
    sgs.CurationV1.insert_curation(
        sorting_id=(
            sgs.MetricCurationSelection
            & {"metric_curation_id": metric_curation_id["metric_curation_id"]}
        ).fetch1("sorting_id"),
        parent_curation_id=0,
        labels=labels,
        merge_groups=merge_groups,
        metrics=metrics,
        description="after metric curation",
    )

[2025-12-20 14:30:25,863][WARNING]: Skipped checksum for file with hash: 49692bf9-7354-b154-d933-931c82797bfb, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_SYBL05AX1C.nwb
[2025-12-20 14:30:47,094][WARNING]: Skipped checksum for file with hash: 49692bf9-7354-b154-d933-931c82797bfb, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_SYBL05AX1C.nwb
[2025-12-20 14:31:08,488][WARNING]: Skipped checksum for file with hash: 49692bf9-7354-b154-d933-931c82797bfb, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_SYBL05AX1C.nwb
[14:31:50][INFO] Spyglass: Writing new NWB file wilbur20210512_VW1W0X6U6W.nwb
[14:33:56][INFO] Spyglass: Writing new NWB file wilbur20210512_79LFQ99ZDM.nwb
[14:36:00][INFO] Spyglass: Writing new NWB file wilbur20210512_THGEP2GY0I.nwb
[2025-12-20 14:36:41,304][WARNING]: Skipped checksum for file with hash: 382b2808-08c5-0f33-ada8-e64bf3a23f72, and pat

In [ ]:
sorting_ids = (sgs.MetricCurationSelection() & metric_curation_ids).fetch("sorting_id")
sorting_ids = [{"sorting_id": sorting_id} for sorting_id in sorting_ids]
curation_keys = (sgs.CurationV1 &  sorting_ids & {"curation_id": 1}).fetch("KEY")
print(len(curation_keys))
for key in curation_keys:
    print("key: ", key)
    curation_uri = sgs.FigURLCurationSelection().generate_curation_uri(
        key
    )
    
    metric_curation_key = (sgs.MetricCurationSelection() & {"sorting_id": key["sorting_id"]}).fetch1("KEY")
    metrics = sgs.MetricCuration.get_metrics(metric_curation_key)
    # print("mc_key: ", metric_curation_key)
    # print("metrics: ", metrics.keys())
    
    fig_selection_key = {
        "sorting_id": key["sorting_id"],
        "curation_id": 1,
        "curation_uri": curation_uri,
        "metrics_figurl": list(metrics.keys())
    }
    sgs.FigURLCurationSelection.insert_selection(fig_selection_key)


sgs.FigURLCurationSelection() & sort_group_ids()

12
key:  {'sorting_id': UUID('0c0d6f10-5af6-4add-beac-67f101ecb017'), 'curation_id': 1}


[2025-12-20 15:02:18,064][WARNING]: Skipped checksum for file with hash: 49692bf9-7354-b154-d933-931c82797bfb, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_SYBL05AX1C.nwb


key:  {'sorting_id': UUID('103990e9-2063-4541-906e-5c69fdf42339'), 'curation_id': 1}
key:  {'sorting_id': UUID('1dd1e911-c185-4488-b607-83502ac5225f'), 'curation_id': 1}
key:  {'sorting_id': UUID('45539c29-3436-4633-b63d-f499fe8e4073'), 'curation_id': 1}


[2025-12-20 15:04:25,110][WARNING]: Skipped checksum for file with hash: 382b2808-08c5-0f33-ada8-e64bf3a23f72, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_R8ZJLVDTDQ.nwb


key:  {'sorting_id': UUID('48a2c037-f872-4a63-9ccb-1bd14cb727e4'), 'curation_id': 1}
key:  {'sorting_id': UUID('6868b312-14db-473b-ae34-ce1bd193055d'), 'curation_id': 1}


...

[2025-12-20 15:07:14,766][WARNING]: Skipped checksum for file with hash: 024e53a0-6dd6-c5f1-2eff-6e6b61376c88, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_P08U9996AP.nwb


key:  {'sorting_id': UUID('d14d9669-4f3e-47c8-bed9-e0cc90be7a71'), 'curation_id': 1}
key:  {'sorting_id': UUID('df857aaa-ccd8-4abf-82e2-e9b97706c162'), 'curation_id': 1}
key:  {'sorting_id': UUID('ee34cd1e-e742-4c73-bdae-9974c2b1d74a'), 'curation_id': 1}


[2025-12-20 15:09:20,653][WARNING]: Skipped checksum for file with hash: e9f68e17-5a4a-fc39-fdac-377f87d6a9ae, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_B5RLR8E8OR.nwb


key:  {'sorting_id': UUID('f3df9cb1-5729-4521-8624-1dd556b3adb2'), 'curation_id': 1}


figurl_curation_id,sorting_id,curation_id,curation_uri GitHub-based URI to a file to which the manual curation will be saved,metrics_figurl metrics to display in the figURL
008637bc-1444-4bcd-9817-a3839b0440c2,298552b8-9ebc-420d-b496-acb310a2e637,1,sha1://b8cedeb3172945d06d206fdde92902f85757896c,=BLOB=
017acf6d-e6b5-4745-b95f-ba76ea09d0bc,fe96651e-adff-4bb3-9a23-0a2cb1eeab1c,1,sha1://f19269e1d7d9c51bdb68014c8852bfcb798ff526,=BLOB=
027c6d58-173d-4b4c-8b9a-9b1f974ef306,103990e9-2063-4541-906e-5c69fdf42339,1,sha1://ae6460dd80ee4770d73a37825541c58f9f4e589d,=BLOB=
186aae3c-b07b-4942-aee1-ff9755e86530,0c0d6f10-5af6-4add-beac-67f101ecb017,1,sha1://488ffe6752c962059dc7292d0cf9d2a519437b22,=BLOB=
194fbf67-78d6-4f25-935f-6b49e7626110,9552f680-911f-40a5-83a1-93990c18ef6b,1,sha1://45459f5b6c5af6cabc9c28393865bc3390d9ec79,=BLOB=
1ceabd73-62cf-4cd6-bfa0-0b9cb086caa9,80021b2e-9bbc-4b87-825e-664ab9996717,1,sha1://e727cf3e3756dc32fdb27bbe514c392b024ddb0e,=BLOB=
250f9bb4-729c-4e97-af2e-be57d0aab0b3,04abb6b4-7cf8-408f-bce0-97baa216e6bd,1,sha1://1d54977afc1485a9d68df2847e87ea8a72b6708f,=BLOB=
3aeb0218-41ad-411f-b350-e7e238fdd363,d14d9669-4f3e-47c8-bed9-e0cc90be7a71,1,sha1://467eb0b260bf7cf91701ec93579b6877171822b4,=BLOB=
437e48cb-c41c-41a3-99e3-5f9fc6f86986,ee34cd1e-e742-4c73-bdae-9974c2b1d74a,1,sha1://17d6453791cf71838bf3d1386896a708963078af,=BLOB=
476a9562-b559-4472-b6b7-de04a1eb9a18,76904362-d49e-4883-98d1-442822ebb9a1,1,sha1://cd0d6b720dddb273a0538cb36045e97fb433eb3a,=BLOB=


In [84]:
sgs.FigURLCurationSelection() & sorting_ids

figurl_curation_id,sorting_id,curation_id,curation_uri GitHub-based URI to a file to which the manual curation will be saved,metrics_figurl metrics to display in the figURL
027c6d58-173d-4b4c-8b9a-9b1f974ef306,103990e9-2063-4541-906e-5c69fdf42339,1,sha1://ae6460dd80ee4770d73a37825541c58f9f4e589d,=BLOB=
186aae3c-b07b-4942-aee1-ff9755e86530,0c0d6f10-5af6-4add-beac-67f101ecb017,1,sha1://488ffe6752c962059dc7292d0cf9d2a519437b22,=BLOB=
3aeb0218-41ad-411f-b350-e7e238fdd363,d14d9669-4f3e-47c8-bed9-e0cc90be7a71,1,sha1://467eb0b260bf7cf91701ec93579b6877171822b4,=BLOB=
437e48cb-c41c-41a3-99e3-5f9fc6f86986,ee34cd1e-e742-4c73-bdae-9974c2b1d74a,1,sha1://17d6453791cf71838bf3d1386896a708963078af,=BLOB=
476a9562-b559-4472-b6b7-de04a1eb9a18,76904362-d49e-4883-98d1-442822ebb9a1,1,sha1://cd0d6b720dddb273a0538cb36045e97fb433eb3a,=BLOB=
7cfe061b-7a59-4922-a7dc-e8214348c450,1dd1e911-c185-4488-b607-83502ac5225f,1,sha1://2822732e4ccbc7491613a038f81654cc5a9a1eb6,=BLOB=
7ff97842-9875-4eb5-8c5b-dcfa14625e55,df857aaa-ccd8-4abf-82e2-e9b97706c162,1,sha1://45459f5b6c5af6cabc9c28393865bc3390d9ec79,=BLOB=
89295583-695a-4c8e-8a0a-972cc6bcad9d,f3df9cb1-5729-4521-8624-1dd556b3adb2,1,sha1://867664b4321a31cc188c58113679c39180fafef8,=BLOB=
9181c507-403b-4281-b885-480c1952dd53,bae249c1-f797-4e38-83ee-a21f51684a20,1,sha1://0f89ac5f3551fa848053318615b08a8400f7ccbc,=BLOB=
9d61d99d-7f30-440f-87aa-f799b60e3077,6868b312-14db-473b-ae34-ce1bd193055d,1,sha1://e080ead142b3eca73de663675efcf1b02fed4358,=BLOB=


In [92]:
keys =  (sgs.FigURLCurationSelection - sgs.FigURLCuration).fetch("KEY")
# sgs.FigURLCuration.populate(keys)
procs = smp.launch_workers((sgs.FigURLCuration), keys, n_workers=6)
for p in procs: p.join()

[2025-12-20 16:16:25,505][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,505][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,506][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,511][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,515][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,521][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,522][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,522][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,523][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 16:16:25,528][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-20 16:16:27,124][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:27,124][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:27,124][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-20 16:16:27,135][INFO]: Connected anirudh@172.16.102.154:330

int_type: <class 'numpy.int32'>
Initial pass: segment 0
Segment 0 of 4
Segment 1 of 4


[16:17:43][INFO] Spyglass: Preparing spikesortingview data
[16:17:43][INFO] Spyglass: Preparing spikesortingview data


int_type: <class 'numpy.int32'>
Initial pass: segment 0
int_type: <class 'numpy.int32'>
Initial pass: segment 0


[16:17:45][INFO] Spyglass: Preparing spikesortingview data
[16:17:45][INFO] Spyglass: Preparing spikesortingview data


int_type: <class 'numpy.int32'>
Initial pass: segment 0
int_type: <class 'numpy.int32'>
Initial pass: segment 0
Segment 2 of 4
Segment 3 of 4
Computing sha1 of /tmp/tmp32pbsemo/spikesortingview.h5


FigURLCuration: 100%|██████████| 1/1 [03:01<00:00, 181.08s/it]


Segment 0 of 4
Segment 0 of 4
Segment 0 of 4
Segment 1 of 4
Segment 1 of 4
Segment 1 of 4
Segment 0 of 4
Segment 1 of 4
Segment 2 of 4
Segment 2 of 4
Segment 2 of 4
Segment 2 of 4
Segment 3 of 4
Segment 3 of 4
Segment 3 of 4
Computing sha1 of /tmp/tmpv5_2bvz0/spikesortingview.h5
Computing sha1 of /tmp/tmp4zlvvqyd/spikesortingview.h5
Computing sha1 of /tmp/tmp87ixhbks/spikesortingview.h5
Segment 3 of 4
Computing sha1 of /tmp/tmpl9r3bapi/spikesortingview.h5


FigURLCuration: 100%|██████████| 1/1 [17:24<00:00, 1044.71s/it]


In [93]:
q = sgs.CurationV1 & {"curation_id": 1} & f"analysis_file_name LIKE '{prefix}\\_%'"

key =(sgs.FigURLCurationSelection() & q).fetch("KEY")


[2025-12-20 17:46:46,957][WARNING]: MySQL server has gone away. Reconnecting to the server.


In [97]:
figurl_curation_ids = (sgs.FigURLCurationSelection() & sorting_ids).fetch("KEY")

In [98]:
sgs.FigURLCuration() & figurl_curation_ids

figurl_curation_id,url
027c6d58-173d-4b4c-8b9a-9b1f974ef306,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:d5274b21772eb0a999a423d67f4ab109214dafd5&s={""initialSortingCuration"":""sha1://ae6460dd80ee4770d73a37825541c58f9f4e589d"",""sortingCuration"":""sha1://ae6460dd80ee4770d73a37825541c58f9f4e589d""}&label=44995126-6ec9-4afa-9799-b46b8d668fe6%20103990e9-2063-4541-906e-5c69fdf42339&zone=default"
186aae3c-b07b-4942-aee1-ff9755e86530,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:d19149ac9a49379a87af33ef18fe061f03099511&s={""initialSortingCuration"":""sha1://488ffe6752c962059dc7292d0cf9d2a519437b22"",""sortingCuration"":""sha1://488ffe6752c962059dc7292d0cf9d2a519437b22""}&label=c2beb7d3-6a19-416c-a469-b2a05e292315%200c0d6f10-5af6-4add-beac-67f101ecb017&zone=default"
3aeb0218-41ad-411f-b350-e7e238fdd363,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:a4382d8eda6e6545fe4663530754c2e057cae7b2&s={""initialSortingCuration"":""sha1://467eb0b260bf7cf91701ec93579b6877171822b4"",""sortingCuration"":""sha1://467eb0b260bf7cf91701ec93579b6877171822b4""}&label=04bfe52f-8cc0-43d9-8105-ad834393dc88%20d14d9669-4f3e-47c8-bed9-e0cc90be7a71&zone=default"
437e48cb-c41c-41a3-99e3-5f9fc6f86986,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:a2fe9aca9306b650ea80639299df2215fd518bc5&s={""initialSortingCuration"":""sha1://17d6453791cf71838bf3d1386896a708963078af"",""sortingCuration"":""sha1://17d6453791cf71838bf3d1386896a708963078af""}&label=d7f8755c-0445-4217-8ca1-e6df2bc3a855%20ee34cd1e-e742-4c73-bdae-9974c2b1d74a&zone=default"
476a9562-b559-4472-b6b7-de04a1eb9a18,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:73824d7bf16e96135c8ad4e8dc03466248d44efa&s={""initialSortingCuration"":""sha1://cd0d6b720dddb273a0538cb36045e97fb433eb3a"",""sortingCuration"":""sha1://cd0d6b720dddb273a0538cb36045e97fb433eb3a""}&label=1c83093e-f751-489c-bdae-674e27b78e75%2076904362-d49e-4883-98d1-442822ebb9a1&zone=default"
7cfe061b-7a59-4922-a7dc-e8214348c450,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:35e3ec3dfe1a5f85e9fc0f6bfeb1c0574db9703e&s={""initialSortingCuration"":""sha1://2822732e4ccbc7491613a038f81654cc5a9a1eb6"",""sortingCuration"":""sha1://2822732e4ccbc7491613a038f81654cc5a9a1eb6""}&label=0270ec87-0bd4-47c6-ba5a-457757971fb7%201dd1e911-c185-4488-b607-83502ac5225f&zone=default"
7ff97842-9875-4eb5-8c5b-dcfa14625e55,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:7eabf26d5795a312bf0713cfa4749f52dc52bd11&s={""initialSortingCuration"":""sha1://45459f5b6c5af6cabc9c28393865bc3390d9ec79"",""sortingCuration"":""sha1://45459f5b6c5af6cabc9c28393865bc3390d9ec79""}&label=41fbc627-282b-4964-b563-e73fb4580bbd%20df857aaa-ccd8-4abf-82e2-e9b97706c162&zone=default"
89295583-695a-4c8e-8a0a-972cc6bcad9d,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:c26044ca987aadf20da280588e5c9894383eab81&s={""initialSortingCuration"":""sha1://867664b4321a31cc188c58113679c39180fafef8"",""sortingCuration"":""sha1://867664b4321a31cc188c58113679c39180fafef8""}&label=6be96d31-2935-4c11-b1ee-7b1f9ccf9169%20f3df9cb1-5729-4521-8624-1dd556b3adb2&zone=default"
9181c507-403b-4281-b885-480c1952dd53,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:c1e7f9d49ef831bc1e0e1e74a60208583afe3278&s={""initialSortingCuration"":""sha1://0f89ac5f3551fa848053318615b08a8400f7ccbc"",""sortingCuration"":""sha1://0f89ac5f3551fa848053318615b08a8400f7ccbc""}&label=8f212513-a138-4ca6-9d5b-f44b3ad42f06%20bae249c1-f797-4e38-83ee-a21f51684a20&zone=default"
9d61d99d-7f30-440f-87aa-f799b60e3077,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:16710c5bd6c431e48eff00c63bc374485e2a0fdb&s={""initialSortingCuration"":""sha1://e080ead142b3eca73de

#### Re-insert into sgs.CurationV1

In [219]:
from __future__ import annotations

import json
import re
from pathlib import Path
from urllib.parse import urlparse, parse_qs, unquote

# --- Parsing ---

_SHA1_URI_RE = r"sha1://[0-9a-f]{40}"

def extract_initial_and_updated_uris(figurl: str) -> tuple[str, str]:
    """
    Extract (initial_uri, updated_uri) from a figurl sortingview link.

    Looks for:
      - initialSortingCuration -> initial_uri
      - sortingCuration        -> updated_uri
    """
    figurl = figurl.strip()
    if not figurl:
        raise ValueError("Empty URL line")

    parsed = urlparse(figurl)
    qs = parse_qs(parsed.query)

    # Preferred: parse the `s=` query param (URL-encoded JSON)
    s_param = qs.get("s", [None])[0]
    if s_param is not None:
        s_decoded = unquote(s_param).strip()

        # Sometimes `s` is JSON; try json.loads first
        try:
            payload = json.loads(s_decoded)
            initial_uri = payload["initialSortingCuration"]
            updated_uri = payload["sortingCuration"]
            return initial_uri, updated_uri
        except Exception:
            # Fall through to regex approach below
            pass

    # Fallback: decode whole URL and regex the JSON fragment
    decoded_all = unquote(figurl)

    m_init = re.search(rf'initialSortingCuration"\s*:\s*"({_SHA1_URI_RE})"', decoded_all)
    m_upd  = re.search(rf'sortingCuration"\s*:\s*"({_SHA1_URI_RE})"', decoded_all)

    if not (m_init and m_upd):
        raise ValueError(f"Could not find both URIs in URL: {figurl}")

    return m_init.group(1), m_upd.group(1)


def iter_figurl_links(path: str | Path):
    path = Path(path)
    for i, line in enumerate(path.read_text().splitlines(), start=1):
        s = line.strip()
        if not s or s.startswith("#"):
            continue
        yield i, s


# --- Processing (your logic) ---

def insert_from_pair(
    *,
    initial_uri: str,
    updated_uri: str,
    parent_curation_id: int = 1,
    apply_merge: bool = False,
    description: str = "after FigURL curation",
):
    sorting_id = (
        sgs.FigURLCurationSelection()
        & {"curation_uri": initial_uri}
        & session_restrict
    ).fetch1("sorting_id")

    labels = sgs.FigURLCuration.get_labels(updated_uri)
    merge_groups = sgs.FigURLCuration.get_merge_groups(updated_uri)

    curation_key = sgs.CurationV1.insert_curation(
        sorting_id=sorting_id,
        parent_curation_id=parent_curation_id,
        labels=labels,
        merge_groups=merge_groups,
        apply_merge=apply_merge,
        description=description,
    )
    return curation_key


# --- Batch runner ---

links_path = "/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wtrack_all_curation_results.txt"  

parent_curation_id = 1
apply_merge = False
description = "after FigURL curation"

dry_run = False  # set True to only print parsed URIs, no insert
skip_duplicates_in_file = True

seen_pairs = set()
results = []
errors = []

for line_no, url in iter_figurl_links(links_path):
    try:
        initial_uri, updated_uri = extract_initial_and_updated_uris(url)

        if skip_duplicates_in_file:
            pair = (initial_uri, updated_uri)
            if pair in seen_pairs:
                print(f"[line {line_no}] duplicate pair in file; skipping")
                continue
            seen_pairs.add(pair)

        print(f"[line {line_no}] initial={initial_uri} updated={updated_uri}")

        if dry_run:
            results.append(
                {"line": line_no, "initial_uri": initial_uri, "updated_uri": updated_uri, "status": "DRY_RUN"}
            )
            continue

        curation_key = insert_from_pair(
            initial_uri=initial_uri,
            updated_uri=updated_uri,
            parent_curation_id=parent_curation_id,
            apply_merge=apply_merge,
            description=description,
        )
        results.append(
            {"line": line_no, "initial_uri": initial_uri, "updated_uri": updated_uri, "status": "OK", "curation_key": curation_key}
        )

    except Exception as e:
        errors.append({"line": line_no, "url": url, "error": repr(e)})
        print(f"[line {line_no}] ERROR: {e!r}")

print(f"Done. OK={sum(r['status']=='OK' for r in results)} DRY_RUN={sum(r['status']=='DRY_RUN' for r in results)} ERRORS={len(errors)}")


[line 1] initial=sha1://ae6460dd80ee4770d73a37825541c58f9f4e589d updated=sha1://d468b1bc665957661dfbdd4dceb38de6575feae5


[18:10:54][INFO] Spyglass: Writing new NWB file wilbur20210512_GX08DROXW9.nwb


[line 3] initial=sha1://488ffe6752c962059dc7292d0cf9d2a519437b22 updated=sha1://3d49b26d1585a628922110f0001d43104bdf0bea


[18:11:55][INFO] Spyglass: Writing new NWB file wilbur20210512_3Y265GDDWO.nwb


[line 5] initial=sha1://467eb0b260bf7cf91701ec93579b6877171822b4 updated=sha1://484e3c1628a7a362cd83d18dd568dc2c99f05d12


...

[line 21] initial=sha1://3801865d4f6844a1e282d977d28b88744e371945 updated=sha1://d86ae26729281a836498d5cc2f76a1168d03d35b


[18:21:22][INFO] Spyglass: Writing new NWB file wilbur20210512_A1S6173JFY.nwb


[line 23] initial=sha1://d8f061d572bc0f0d778cee1180a6c06ea43a8698 updated=sha1://68b3019b23f32e95cc69ddbcb4c4d90b8d2ee544


[18:22:24][INFO] Spyglass: Writing new NWB file wilbur20210512_XTTXDC6JUB.nwb


Done. OK=12 DRY_RUN=0 ERRORS=0


In [218]:
epochwise_rec_ids = (sgs.SpikeSortingRecordingSelection() & session_restrict & 'interval_list_name = "wtrack_all"').fetch("recording_id")
epochwise_rec_keys = [{"recording_id": rid} for rid in epochwise_rec_ids]
epochwise_sort_keys = (sgs.SpikeSortingSelection & epochwise_rec_keys).fetch("KEY")
(sgs.CurationV1()  & epochwise_sort_keys) & {"parent_curation_id": 0, "curation_id": 2}

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description


In [209]:
ten_sort_ids = ((sgs.CurationV1()  & epochwise_sort_keys) & {"parent_curation_id": 0, "curation_id": 2}).fetch("sorting_id")
ten_sort_keys = [{"sorting_id": key} for key in ten_sort_ids]
ten_rec_ids = (sgs.SpikeSortingSelection() & ten_sort_keys).fetch("recording_id")
ten_rec_keys = [{"recording_id": key} for key in ten_rec_ids]
sgs.SpikeSortingRecordingSelection & ten_rec_keys

recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name


In [215]:
twelve_sort_ids = ((sgs.CurationV1()  & epochwise_sort_keys) & {"parent_curation_id": 0, "curation_id": 1}).fetch("sorting_id")
twelve_sort_keys = [{"sorting_id": key} for key in twelve_sort_ids]
twelve_rec_ids = (sgs.SpikeSortingSelection() & twelve_sort_keys).fetch("recording_id")
twelve_rec_keys = [{"recording_id": key} for key in twelve_rec_ids]
sgs.SpikeSortingRecordingSelection & twelve_rec_keys

recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name
0270ec87-0bd4-47c6-ba5a-457757971fb7,wilbur20210512_.nwb,33,wtrack_all,default,NeuroACT
04bfe52f-8cc0-43d9-8105-ad834393dc88,wilbur20210512_.nwb,30,wtrack_all,default,NeuroACT
1c83093e-f751-489c-bdae-674e27b78e75,wilbur20210512_.nwb,29,wtrack_all,default,NeuroACT
41fbc627-282b-4964-b563-e73fb4580bbd,wilbur20210512_.nwb,34,wtrack_all,default,NeuroACT
44995126-6ec9-4afa-9799-b46b8d668fe6,wilbur20210512_.nwb,35,wtrack_all,default,NeuroACT
6be96d31-2935-4c11-b1ee-7b1f9ccf9169,wilbur20210512_.nwb,31,wtrack_all,default,NeuroACT
8f212513-a138-4ca6-9d5b-f44b3ad42f06,wilbur20210512_.nwb,26,wtrack_all,default,NeuroACT
c2beb7d3-6a19-416c-a469-b2a05e292315,wilbur20210512_.nwb,25,wtrack_all,default,NeuroACT
d7f8755c-0445-4217-8ca1-e6df2bc3a855,wilbur20210512_.nwb,27,wtrack_all,default,NeuroACT
e50cbee4-ada1-498b-a9df-c2883e03e180,wilbur20210512_.nwb,32,wtrack_all,default,NeuroACT


In [221]:
(sgs.CurationV1() & f"analysis_file_name LIKE '{prefix}\\_%'") & {"description": "after FigURL curation"}

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
0c0d6f10-5af6-4add-beac-67f101ecb017,2,1,wilbur20210512_3Y265GDDWO.nwb,909d3804-abf4-4bfa-9acc-53a7eaf7e6bd,0,after FigURL curation
103990e9-2063-4541-906e-5c69fdf42339,2,1,wilbur20210512_GX08DROXW9.nwb,ea33f22f-21ae-444a-929c-9af648e5262c,0,after FigURL curation
1dd1e911-c185-4488-b607-83502ac5225f,2,1,wilbur20210512_G7CD0M7VE8.nwb,527b0b67-7b40-4d73-b8ed-4c30c385b0b8,0,after FigURL curation
45539c29-3436-4633-b63d-f499fe8e4073,2,1,wilbur20210512_XTTXDC6JUB.nwb,882b6b9b-336d-49be-9e0d-aaafe957148f,0,after FigURL curation
48a2c037-f872-4a63-9ccb-1bd14cb727e4,2,1,wilbur20210512_A1S6173JFY.nwb,dd032fe7-496a-4772-9641-4ab4abf653d7,0,after FigURL curation
6868b312-14db-473b-ae34-ce1bd193055d,2,1,wilbur20210512_0S1673QTKH.nwb,a5d922a7-1dd7-44b2-8db6-2aa4c3c30f88,0,after FigURL curation
76904362-d49e-4883-98d1-442822ebb9a1,2,1,wilbur20210512_HLCGRU1TYF.nwb,a9ced799-592d-45f8-84e8-d764a53f55b6,0,after FigURL curation
bae249c1-f797-4e38-83ee-a21f51684a20,2,1,wilbur20210512_O73UR4N6AW.nwb,2c664ca0-f8a9-4d0b-8b01-8ef1030cdac4,0,after FigURL curation
d14d9669-4f3e-47c8-bed9-e0cc90be7a71,2,1,wilbur20210512_9KMH0SM1VF.nwb,97dd707e-49e4-4ae4-89ec-94b089bf9d4a,0,after FigURL curation
df857aaa-ccd8-4abf-82e2-e9b97706c162,2,1,wilbur20210512_B2C5M38SSS.nwb,612ee4e8-1a41-4c06-9ff4-f85c6f8b767e,0,after FigURL curation


In [122]:
# DO THIS FOR EVERY PAIR OF initial_uri, updated_uri
initial_uri = "sha1://45459f5b6c5af6cabc9c28393865bc3390d9ec79"
updated_uri = "sha1://c72c029d3449a035b88849896bff949492294dd7"

sorting_id = (sgs.FigURLCurationSelection() &  {"curation_uri":initial_uri} & session_restrict).fetch1("sorting_id")
parent_curation_id = 1

labels = sgs.FigURLCuration.get_labels(updated_uri)
merge_groups = sgs.FigURLCuration.get_merge_groups(updated_uri)

curation_key = sgs.CurationV1.insert_curation(
    sorting_id=sorting_id,
    parent_curation_id= parent_curation_id,
    labels=labels,
    merge_groups=merge_groups,
    apply_merge=False,
    description= "after FigURL curation"
)


DataJointError: fetch1 should only return one tuple. 2 tuples found

#### Find merge candidates

In [3]:
import spikeinterface.qualitymetrics as sqm
from spikeinterface.postprocessing import compute_correlograms

figurl_curation_ids = (
    sgs.CurationV1
    & f"analysis_file_name LIKE '{prefix}\\_%'"
    & {"description": "after FigURL curation"}
).fetch("KEY", as_dict=True)

len(figurl_curation_ids)


12

In [267]:
figurl_curation_key = figurl_curation_ids[11]

In [257]:
recording = sgs.CurationV1.get_recording(figurl_curation_key)
sorting = sgs.CurationV1.get_sorting(figurl_curation_key)

print("n_units:", len(sorting.get_unit_ids()))
print("fs:", recording.get_sampling_frequency())

waveforms_folder = os.path.join(base_dir, "tmp", "waveforms_for_merging", str(figurl_curation_key["sorting_id"]))
os.makedirs(waveforms_folder, exist_ok=True)

we = si.extract_waveforms(
    recording=recording,
    sorting=sorting,
    folder=waveforms_folder,
    ms_before=0.5,
    ms_after=0.5,
    max_spikes_per_unit=2000,
    overwrite=True,     
    return_scaled=True,
    sparse=True,
    n_jobs=16,
)

print("WaveformExtractor unit_ids:", len(we.unit_ids))


[2025-12-23 15:33:47,157][WARNING]: Skipped checksum for file with hash: 7c0d0b44-2a9c-3b78-ab11-f74b25a3251f, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_YF9UMEI5Z0.nwb
[2025-12-23 15:34:06,408][WARNING]: Skipped checksum for file with hash: 7c0d0b44-2a9c-3b78-ab11-f74b25a3251f, and path: /media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_YF9UMEI5Z0.nwb


n_units: 44
fs: 29959.314285714285


extract waveforms shared_memory multi buffer:   0%|          | 0/3637 [00:00<?, ?it/s]

extract waveforms memmap multi buffer:   0%|          | 0/3637 [00:00<?, ?it/s]

WaveformExtractor unit_ids: 44


In [258]:
isi_res = sqm.compute_isi_violations(
    we,
    isi_threshold_ms=1.5,  
    min_isi_ms=0.0,
)

isi_ratio_by_unit = isi_res.isi_violations_ratio  
isi_count_by_unit = isi_res.isi_violations_count  

unit_ids = list(we.unit_ids)
unit_summary = pd.DataFrame(
    {
        "unit_id": unit_ids,
        "isi_violation_ratio": [isi_ratio_by_unit.get(u, np.nan) for u in unit_ids],
        "isi_violation_count": [isi_count_by_unit.get(u, np.nan) for u in unit_ids],
    }
)

unit_summary.sort_values("isi_violation_ratio").head(10)

,unit_id,isi_violation_ratio,isi_violation_count
17,18,0.000000,0
32,33,0.000000,0
33,34,0.000000,0
2,3,0.000638,9
19,20,0.002449,1
35,36,0.002902,9
16,17,0.008726,2
42,43,0.010962,116
15,16,0.023699,20
43,44,0.024728,10


In [259]:
templates = we.get_all_templates()  
n_units = templates.shape[0]

templates_flat = templates.reshape(n_units, -1)
norms = np.linalg.norm(templates_flat, axis=1, keepdims=True)
templates_unit = templates_flat / np.clip(norms, 1e-12, None)

cos_sim = templates_unit @ templates_unit.T  
np.fill_diagonal(cos_sim, np.nan)  

cos_sim.shape


(44, 44)

In [260]:
ccgs_100, bins_100 = compute_correlograms(
    sorting,
    window_ms=200.0,  
    bin_ms=1.0,
    method="auto",
)

bin_centers_100 = (bins_100[:-1] + bins_100[1:]) / 2
zero_bin_idx_100 = int(np.argmin(np.abs(bin_centers_100)))

unit_to_index = {u: i for i, u in enumerate(sorting.get_unit_ids())}

(ccgs_100.shape, bins_100.shape, zero_bin_idx_100, bin_centers_100[zero_bin_idx_100])


((44, 44, 198), (199,), 98, -0.5006790161132812)

In [261]:
def ccg_pair_counts_and_asymmetry(ccg_counts: np.ndarray, zero_idx: int, centers: np.ndarray):
    """
    Implements the asymmetry metric.

    - Splits the correlogram into negative side, zero bin, positive side.
    - Computes:
        total = neg + zero + pos
        dominant_side = max(neg, pos)
        ratio = (dominant_side + 0.5 * zero) / total
    """
    neg_sum = float(ccg_counts[centers < 0].sum())
    pos_sum = float(ccg_counts[centers > 0].sum())
    zero = float(ccg_counts[zero_idx])

    total = neg_sum + pos_sum + zero
    if total <= 0:
        return dict(total=0.0, neg_sum=neg_sum, pos_sum=pos_sum, zero=zero, asym_ratio=np.nan)

    asym_ratio = (max(neg_sum, pos_sum) + 0.5 * zero) / total
    return dict(total=total, neg_sum=neg_sum, pos_sum=pos_sum, zero=zero, asym_ratio=asym_ratio)


def which_unit_leads_from_ccg(ccg_counts: np.ndarray, centers: np.ndarray, unit_a, unit_b):
    """
    Decide which unit tends to spike first.

    IMPORTANT: SpikeInterface defines ccg[A,B] as histogram of (t_A - t_B).

    If A tends to spike first, then t_A < t_B, so (t_A - t_B) is NEGATIVE.
    Therefore: more mass on the negative side => A leads.
    """
    neg_sum = float(ccg_counts[centers < 0].sum())
    pos_sum = float(ccg_counts[centers > 0].sum())

    if neg_sum > pos_sum:
        return unit_a
    elif pos_sum > neg_sum:
        return unit_b
    else:
        return None  

def template_peak_to_peak_uV(we, unit_id):
    template = we.get_template(unit_id)
    ptp_per_chan = template.ptp(axis=0) 
    return float(ptp_per_chan.max())




In [262]:
ptp_by_unit = {u: template_peak_to_peak_uV(we, u) for u in unit_ids}

ptp_by_unit

{1: 49.76217269897461,
 2: 40.198970794677734,
 3: 81.18821716308594,
 4: 34.88658142089844,
 5: 39.463653564453125,
 6: 41.707584381103516,
 7: 47.056427001953125,
 8: 39.70295333862305,
 9: 52.09162902832031,
 10: 58.21361541748047,
 11: 46.52609634399414,
 12: 86.28933715820312,
 13: 42.52137756347656,
 14: 36.210205078125,
 15: 80.06127166748047,
 16: 169.7610321044922,
 17: 195.202392578125,
 18: 284.0085754394531,
 19: 44.65633010864258,
 20: 130.42929077148438,
 21: 199.7644500732422,
 22: 42.59075927734375,
 23: 42.428340911865234,
 24: 37.60662841796875,
 25: 36.12493133544922,
 26: 54.79490661621094,
 27: 45.35821533203125,
 28: 52.9503288269043,
 29: 32.51679611206055,
 30: 37.57783889770508,
 31: 53.40460205078125,
 32: 65.9956283569336,
 33: 128.64114379882812,
 34: 139.29910278320312,
 35: 58.34212112426758,
 36: 104.78730010986328,
 37: 61.02427673339844,
 38: 57.09767150878906,
 39: 46.025306701660156,
 40: 57.75653076171875,
 41: 110.57865905761719,
 42: 69.70206451416

In [263]:
SIM_THRESH_AUTO = 0.6
ISI_THRESH = 0.1
MIN_CCG_COUNT = 100
ASYM_THRESH = 0.6

rows = []

for ia, ua in enumerate(unit_ids):
    for ib, ub in enumerate(unit_ids):
        if ib <= ia:
            continue

        # cosine similarity
        sim = float(cos_sim[ia, ib])
        if not (sim > SIM_THRESH_AUTO):
            continue

        # isi filter
        isi_a = float(isi_ratio_by_unit.get(ua, np.nan))
        isi_b = float(isi_ratio_by_unit.get(ub, np.nan))
        if not (isi_a < ISI_THRESH and isi_b < ISI_THRESH):
            continue

        # cross-correlogram features
        i_sort_a = unit_to_index[ua]
        i_sort_b = unit_to_index[ub]
        ccg_ab = ccgs_100[i_sort_a, i_sort_b, :]  # histogram of (t_ua - t_ub)

        ccg_stats = ccg_pair_counts_and_asymmetry(ccg_ab, zero_bin_idx_100, bin_centers_100)
        if not (ccg_stats["total"] >= MIN_CCG_COUNT and ccg_stats["asym_ratio"] > ASYM_THRESH):
            continue

        # leader vs follower
        leader = which_unit_leads_from_ccg(ccg_ab, bin_centers_100, ua, ub)
        amp_a = ptp_by_unit[ua]
        amp_b = ptp_by_unit[ub]

        # criterion 4: leader must have larger amplitude
        if leader == ua:
            leader_amp_ok = amp_a > amp_b
        elif leader == ub:
            leader_amp_ok = amp_b > amp_a
        else:
            leader_amp_ok = False

        if not leader_amp_ok:
            continue

        rows.append(
            {
                "unit_a": ua,
                "unit_b": ub,
                "cosine_similarity": sim,
                "isi_a": isi_a,
                "isi_b": isi_b,
                "ccg_total_100ms": ccg_stats["total"],
                "ccg_asym_ratio_100ms": ccg_stats["asym_ratio"],
                "leader": leader,
                "ptp_a": amp_a,
                "ptp_b": amp_b,
            }
        )




In [264]:
rows

[]

In [265]:
candidates_auto = pd.DataFrame(rows).sort_values(["cosine_similarity", "ccg_asym_ratio_100ms"], ascending=False)
candidates_auto.head(20)

KeyError: 'cosine_similarity'

In [223]:

def normalize_unit_id(unit_id):
    """
    Convert unit_id into the exact kind of value SpikeInterface expects for filenames.

    Why:
      - pandas sometimes converts integer IDs to floats (e.g., 23 -> 23.0)
      - SpikeInterface uses the string form in filenames (waveforms_{unit_id}.npy)
      - 23 and 23.0 produce different filenames, so 23.0 breaks file lookup

    Rules:
      - numpy int / python int -> int
      - float that is actually an integer (23.0) -> int(23)
      - digit-only strings -> int
      - otherwise return as-is
    """
    if isinstance(unit_id, (int, np.integer)):
        return int(unit_id)

    if isinstance(unit_id, float) and np.isfinite(unit_id) and unit_id.is_integer():
        return int(unit_id)

    if isinstance(unit_id, str) and unit_id.isdigit():
        return int(unit_id)

    return unit_id


def amplitude_vector_for_unit(we, unit_id, max_spikes=3000):
    """
    Build a per-spike amplitude vector for one unit using extracted waveforms.

    Steps:
      1) Normalize unit_id (avoid 23.0 vs 23 filename mismatch)
      2) Load waveforms for that unit: (n_spikes, n_samples, n_channels)
      3) Pick a single channel to represent amplitude:
           - choose the channel with max template peak-to-peak for that unit
      4) For each spike waveform, compute peak-to-peak amplitude on that channel
    """
    unit_id = normalize_unit_id(unit_id)

    wfs = we.get_waveforms(unit_id)  # raises if file missing

    if wfs.shape[0] > max_spikes:
        wfs = wfs[:max_spikes]

    template = we.get_template(unit_id)  # (n_samples, n_channels)
    peak_chan = int(np.argmax(template.ptp(axis=0)))

    amps = wfs[:, :, peak_chan].ptp(axis=1)
    return amps.astype(float)


def density_overlap_hist(x, y, bins=50):
    """
    Approximate distribution overlap in [0, 1] using histogram densities.

    overlap = integral min(p(x), q(x)) dx

    Returns:
      - ~1.0 if distributions are very similar
      - ~0.0 if they barely overlap
    """
    x = np.asarray(x)
    y = np.asarray(y)
    lo = float(np.nanmin([x.min(), y.min()]))
    hi = float(np.nanmax([x.max(), y.max()]))

    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        return np.nan

    edges = np.linspace(lo, hi, bins + 1)
    hx, _ = np.histogram(x, bins=edges, density=True)
    hy, _ = np.histogram(y, bins=edges, density=True)
    bin_w = edges[1] - edges[0]

    return float(np.sum(np.minimum(hx, hy)) * bin_w)


def plot_pair_diagnostics(we, sorting, ua, ub, ccgs, bins, unit_to_index):
    """
    Plot:
      1) Template overlay (on unit A's strongest channel)
      2) Cross-correlogram for the pair
      3) Amplitude density overlap

    Also returns computed stats (overlap, leader, ccg asymmetry).
    """
    ua = normalize_unit_id(ua)
    ub = normalize_unit_id(ub)

    template_a = we.get_template(ua)
    template_b = we.get_template(ub)

    ch = int(np.argmax(template_a.ptp(axis=0)))

    centers = (bins[:-1] + bins[1:]) / 2
    zero_idx = int(np.argmin(np.abs(centers)))

    i_a = unit_to_index[ua]
    i_b = unit_to_index[ub]
    ccg_ab = ccgs[i_a, i_b, :]

    stats = ccg_pair_counts_and_asymmetry(ccg_ab, zero_idx, centers)
    leader = which_unit_leads_from_ccg(ccg_ab, centers, ua, ub)

    amps_a = amplitude_vector_for_unit(we, ua)
    amps_b = amplitude_vector_for_unit(we, ub)
    overlap = density_overlap_hist(amps_a, amps_b, bins=60)

    fig, axs = plt.subplots(1, 3, figsize=(18, 4))

    axs[0].plot(template_a[:, ch], label=f"{ua} (ch {ch})")
    axs[0].plot(template_b[:, ch], label=f"{ub} (ch {ch})")
    axs[0].set_title("Templates overlay")
    axs[0].legend()

    axs[1].bar(centers, ccg_ab, width=(bins[1] - bins[0]), align="center")
    axs[1].set_title(
        f"CCG (t_{ua} - t_{ub})\n"
        f"total={stats['total']:.0f} asym={stats['asym_ratio']:.3f} leader={leader}"
    )
    axs[1].set_xlabel("lag (ms)")

    axs[2].hist(amps_a, bins=60, alpha=0.5, density=True, label=str(ua))
    axs[2].hist(amps_b, bins=60, alpha=0.5, density=True, label=str(ub))
    axs[2].set_title(f"Amplitude densities\noverlap={overlap:.3f}")
    axs[2].legend()

    plt.tight_layout()
    plt.show()

    return {"overlap": overlap, "leader": leader, **stats}


In [256]:
if len(candidates_auto):
    r = candidates_auto.iloc[0]
    plot_pair_diagnostics(we, sorting, r.unit_a, r.unit_b, ccgs_100, bins_100, unit_to_index)


In [225]:
import spikeinterface.postprocessing as spost

spost.compute_spike_amplitudes(
    we,
    peak_sign="neg",
    outputs="by_unit",
    # overwrite=True,
)

amps_ext = (we.get_extension("spike_amplitudes")
            if hasattr(we, "get_extension")
            else we.load_extension("spike_amplitudes"))

amps_by_unit = amps_ext.get_data(outputs="by_unit")


extract amplitudes:   0%|          | 0/3637 [00:00<?, ?it/s]

In [255]:

fs = recording.get_sampling_frequency()

unit_id_1 = 6
unit_id_2 = 7

spike_times_1 = sorting.get_unit_spike_train(unit_id=unit_id_1) / fs
spike_times_2 = sorting.get_unit_spike_train(unit_id=unit_id_2) / fs

amps_1 = amps_by_unit[0][unit_id_1]
amps_2 = amps_by_unit[0][unit_id_2]

plt.figure(figsize=(10, 4))
plt.scatter(spike_times_1, amps_1, s=15, alpha=0.6, label=f"Unit {unit_id_1}")
plt.scatter(spike_times_2, amps_2, s=15, alpha=0.6, label=f"Unit {unit_id_2}")
plt.xlabel("Time (s)")
plt.ylabel("Spike amplitude (uV or recording units)")
plt.title("Spike amplitudes over time (2 units)")
plt.legend(markerscale=4)
plt.tight_layout()
plt.show()


ValueError: x and y must be the same size

In [227]:
len(spike_times_1)

1417

In [228]:
len(spike_times_2)

19706

In [78]:
def pairs_to_merge_groups(pairs):
    """
    Convert a list of (unit_a, unit_b) pairs into merge groups using union-find.
    This handles transitive closure: if (1,2) and (2,3) are selected, group is [1,2,3].
    """
    parent = {}

    def find(x):
        parent.setdefault(x, x)
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra != rb:
            parent[rb] = ra

    for a, b in pairs:
        union(int(a), int(b))

    groups = {}
    for x in parent:
        rx = find(x)
        groups.setdefault(rx, []).append(x)

    merge_groups = [sorted(g) for g in groups.values() if len(g) >= 2]
    return merge_groups


In [79]:
pairs_to_merge = [
    # (unit_a, unit_b),
    # fill this in based on what you visually confirm
]

merge_groups = pairs_to_merge_groups(pairs_to_merge)
merge_groups


[]

In [ ]:
sorting_id = parent_key["sorting_id"]
parent_curation_id = parent_key["curation_id"]

curation_key_new = sgs.CurationV1.insert_curation(
    sorting_id=sorting_id,
    parent_curation_id=parent_curation_id,
    merge_groups=merge_groups,
    apply_merge=False,  # keep merges recorded but not applied yet
    description="auto merge candidates (waveform+ISI+CCG criteria)",
)

curation_key_new


#### Automatic curation (again)

In [278]:
sgs.CurationV1() & f"analysis_file_name LIKE '{prefix}\\_%'"

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
0c0d6f10-5af6-4add-beac-67f101ecb017,0,-1,wilbur20210512_S7W1M5S6FA.nwb,f6fc64a0-430a-4bc1-ace2-b0cd56bff812,0,
0c0d6f10-5af6-4add-beac-67f101ecb017,1,0,wilbur20210512_1EKAWFBB6G.nwb,751d5f5e-6806-43c0-bf4e-4c9716933f39,0,after metric curation
0c0d6f10-5af6-4add-beac-67f101ecb017,2,1,wilbur20210512_3Y265GDDWO.nwb,909d3804-abf4-4bfa-9acc-53a7eaf7e6bd,0,after FigURL curation
103990e9-2063-4541-906e-5c69fdf42339,0,-1,wilbur20210512_8HRX3PVODJ.nwb,47007139-078a-4bc5-88f0-005a7eb2786b,0,
103990e9-2063-4541-906e-5c69fdf42339,1,0,wilbur20210512_WXLPWTR5MT.nwb,a0987f34-708a-4fd5-9be6-2fef6cb6e5ff,0,after metric curation
103990e9-2063-4541-906e-5c69fdf42339,2,1,wilbur20210512_GX08DROXW9.nwb,ea33f22f-21ae-444a-929c-9af648e5262c,0,after FigURL curation
1dd1e911-c185-4488-b607-83502ac5225f,0,-1,wilbur20210512_H6OC9EPRN3.nwb,e1800d28-2b21-49cc-bbb4-ad8612f9f35d,0,
1dd1e911-c185-4488-b607-83502ac5225f,1,0,wilbur20210512_PW83SP8RGM.nwb,2e2b2094-ff53-4543-9b4f-f31d924707c9,0,after metric curation
1dd1e911-c185-4488-b607-83502ac5225f,2,1,wilbur20210512_G7CD0M7VE8.nwb,527b0b67-7b40-4d73-b8ed-4c30c385b0b8,0,after FigURL curation
45539c29-3436-4633-b63d-f499fe8e4073,0,-1,wilbur20210512_FDK5EKGDLH.nwb,0572b81d-4a82-4df4-83ee-5b0ddee9ef2f,0,


In [273]:
label_params = {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']],
                'isi_violation': ['>', 0.0025, ['noise', 'reject']],
                'nn_isolation': ['<', 0.95, ['noise', 'reject']],
                'Firing rate (Hz)': ['<', 0.1, ['noise', 'reject']],
                'snr': ['<', 1.5, ['noise', 'reject']]}

merge_params = {}

sgs.MetricCurationParameters().insert1({
    "metric_curation_param_name": "pfc_round_2",
    "label_params": label_params,
    "merge_params": merge_params
}, skip_duplicates = True)

In [284]:
rec_keys = (sgs.SpikeSortingRecordingSelection() & session_restrict & {"interval_list_name": "wtrack_all"}).fetch("KEY")
sorting_ids = (sgs.SpikeSortingSelection & rec_keys).fetch("KEY")

In [5]:
rec_keys = (sgs.SpikeSortingRecordingSelection() & session_restrict & {"interval_list_name": "wtrack_all"}).fetch("KEY")
sorting_ids = (sgs.SpikeSortingSelection & rec_keys).fetch("KEY")

for sorting_idx in sorting_ids:
    key = {
        "sorting_id": sorting_idx["sorting_id"],
        "curation_id": 2,
        "waveform_param_name": "default_whitened",
        "metric_param_name": "franklab_default",
        "metric_curation_param_name": "pfc_round_2",
    }
    sgs.MetricCurationSelection.insert_selection(key)
sgs.MetricCurationSelection()

[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.
[19:11:30][WARNING] Spyglass: This row has already been inserted.


metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
02480e53-2868-4901-b26d-66e48f583d53,4f9c8ea6-bffb-476d-8908-a14f3fbe5aa7,0,default_whitened,franklab_default,noise_overlap_0.03
045708cb-a7e7-4f2e-8930-e376376dfb80,f3df9cb1-5729-4521-8624-1dd556b3adb2,0,default_whitened,franklab_default,noise_overlap_0.03
07b98aa8-2003-4f04-bab9-4c5085fa893a,430212fc-4265-480f-bed3-45387d7272d9,0,default_whitened,franklab_default,noise_overlap_0.03
0ec9b6d9-6f72-4121-9d55-9f8b90d24f04,7d115bb4-fdd3-4701-8369-e67eb32475ab,0,default_whitened,franklab_default,noise_overlap_0.03
10f68a83-111d-43c2-8b59-894ad803b324,76904362-d49e-4883-98d1-442822ebb9a1,2,default_whitened,franklab_default,pfc_round_2
117397f5-9d53-4923-b2d7-3651d116e931,589b0837-0196-41b2-984c-7f9855716898,0,default_whitened,franklab_default,noise_overlap_0.03
173402d5-01d7-473a-bb9f-6c2c8aaaf6e5,d83385c9-1e7c-480b-a3e0-58506057b775,0,default_whitened,franklab_default,noise_overlap_0.03
1e159fbe-72e7-46c6-9edf-6d41c38b6c13,b0482b2a-33b6-4964-b735-854c54cf6980,0,default_whitened,franklab_default,noise_overlap_0.03
1f8c96e4-bea2-4e57-aa5a-075fda265d83,f36656df-8942-4583-8a0e-f152dd36bc4e,2,default_whitened,franklab_default,pfc_round_2
2132d139-bb74-45e9-ab06-07e591b7be90,f3ca2fee-2b0a-4277-804f-f58b1dc0c3aa,0,default_whitened,franklab_default,noise_overlap_0.03


In [290]:
sgs.MetricCurationSelection() & sorting_ids & {"curation_id": 2}

metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
10f68a83-111d-43c2-8b59-894ad803b324,76904362-d49e-4883-98d1-442822ebb9a1,2,default_whitened,franklab_default,pfc_round_2
5a34ae36-f907-4e37-8cd5-10330c3b3697,d14d9669-4f3e-47c8-bed9-e0cc90be7a71,2,default_whitened,franklab_default,pfc_round_2
7c020084-fa8d-4d10-8cd0-eec51992008e,0c0d6f10-5af6-4add-beac-67f101ecb017,2,default_whitened,franklab_default,pfc_round_2
7c11745d-b970-4d42-8f7d-fa62ce1cd908,ee34cd1e-e742-4c73-bdae-9974c2b1d74a,2,default_whitened,franklab_default,pfc_round_2
94946639-6b8a-4fce-bb0c-a1c4722522ec,1dd1e911-c185-4488-b607-83502ac5225f,2,default_whitened,franklab_default,pfc_round_2
9bc30255-9c92-4ac3-8732-07564cfe9504,df857aaa-ccd8-4abf-82e2-e9b97706c162,2,default_whitened,franklab_default,pfc_round_2
a3d8f0cf-a564-4030-ad29-6a0a5a8a3ba4,bae249c1-f797-4e38-83ee-a21f51684a20,2,default_whitened,franklab_default,pfc_round_2
c45c52c4-c86a-4ab7-b0b6-af7c5e175e6c,45539c29-3436-4633-b63d-f499fe8e4073,2,default_whitened,franklab_default,pfc_round_2
d073da79-5d34-4da9-8090-90f5f786ac63,48a2c037-f872-4a63-9ccb-1bd14cb727e4,2,default_whitened,franklab_default,pfc_round_2
e4a8d1fb-ccd9-4342-8dd3-1e95356d3f1e,103990e9-2063-4541-906e-5c69fdf42339,2,default_whitened,franklab_default,pfc_round_2


In [4]:
mkey = (sgs.MetricCurationSelection() -sgs.MetricCuration()).fetch("KEY")

sid = (sgs.MetricCurationSelection & mkey).fetch("sorting_id")
skey = [{"sorting_id": id} for id in sid]
rid = (sgs.SpikeSortingSelection() & skey).fetch("recording_id")
rkey = [{"recording_id": id} for id in rid]
sgs.SpikeSortingRecordingSelection & rkey

recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name


In [27]:
module_dir = os.getcwd()         
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)

# metric_curation_keys = (sgs.MetricCurationSelection() & sorting_ids & {"curation_id": 2}).fetch("KEY")
mkey = (sgs.MetricCurationSelection() -sgs.MetricCuration()).fetch("KEY")

procs = smp.launch_workers((sgs.MetricCuration), mkey, n_workers=7)
for p in procs: p.join()

[2025-12-23 19:24:25,999][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,001][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,023][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,029][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,045][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,061][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,091][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,107][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,120][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,131][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,132][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,139][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,142][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-23 19:24:26,150][INFO]: Connected anirudh@172.16.102.154:3306

Failed: {'metric_curation_id': UUID('7c020084-fa8d-4d10-8cd0-eec51992008e')} Waveforms have been deleted!


[21:27:35][INFO] Spyglass: Applying curation...
[21:27:35][INFO] Spyglass: Saving to NWB...
[21:28:51][INFO] Spyglass: Writing new NWB file wilbur20210512_7LE45QZ4AJ.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration: 100%|██████████| 1/1 [2:06:17<00:00, 7577.88s/it]
[22:17:08][INFO] Spyglass: Applying curation...
[22:17:08][INFO] Spyglass: Saving to NWB...
[22:17:34][INFO] Spyglass: Writing new NWB file wilbur20210512_CMNNQ5KE3X.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration: 

Failed: {'metric_curation_id': UUID('c45c52c4-c86a-4ab7-b0b6-af7c5e175e6c')} ("Duplicate entry '\\xC4\\R\\xC4\\xC8jJ\\xB7\\xB0\\xB6\\xAF|^\\x17^l' for key '__metric_curation.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')


[22:20:14][INFO] Spyglass: Applying curation...
[22:20:14][INFO] Spyglass: Saving to NWB...
[22:20:33][INFO] Spyglass: Writing new NWB file wilbur20210512_FH6IGEIED8.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration: 100%|██████████| 1/1 [2:57:05<00:00, 10625.80s/it]
[22:57:35][INFO] Spyglass: Applying curation...
[22:57:35][INFO] Spyglass: Saving to NWB...
[22:57:51][INFO] Spyglass: Writing new NWB file wilbur20210512_VC9GD7EIEQ.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration:

Failed: {'metric_curation_id': UUID('10f68a83-111d-43c2-8b59-894ad803b324')} ("Duplicate entry '\\x10\\xF6\\x8A\\x83\\x11\\x1DC\\xC2\\x8BY\\x89J\\xD8\\x03\\xB3$' for key '__metric_curation.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')


[23:01:56][INFO] Spyglass: Applying curation...
[23:01:56][INFO] Spyglass: Saving to NWB...
[23:02:11][INFO] Spyglass: Writing new NWB file wilbur20210512_L7S7KZMY09.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration:   0%|          | 0/1 [3:38:37<?, ?it/s]


Failed: {'metric_curation_id': UUID('e669ece1-52e2-482b-88de-17a52e26de7d')} ("Duplicate entry '\\xE6i\\xEC\\xE1R\\xE2H+\\x88\\xDE\\x17\\xA5.&\\xDE}' for key '__metric_curation.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')


[23:09:59][INFO] Spyglass: Applying curation...
[23:09:59][INFO] Spyglass: Saving to NWB...
[23:10:13][INFO] Spyglass: Writing new NWB file wilbur20210512_YONGIIGX71.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/build/objectmapper.py:267: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
MetricCuration:   0%|          | 0/1 [3:46:38<?, ?it/s]


Failed: {'metric_curation_id': UUID('a3d8f0cf-a564-4030-ad29-6a0a5a8a3ba4')} ("Duplicate entry '\\xA3\\xD8\\xF0\\xCF\\xA5d@0\\xAD)j\\x0AZ\\x8A;\\xA4' for key '__metric_curation.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')


In [6]:
sgs.MetricCuration() & f"analysis_file_name LIKE '{prefix}\\_%'"

metric_curation_id,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
045708cb-a7e7-4f2e-8930-e376376dfb80,wilbur20210512_GU99WD3YK1.nwb,c27d8560-f44e-4e05-961c-2688051f8083
10f68a83-111d-43c2-8b59-894ad803b324,wilbur20210512_Y7PBU0R4KA.nwb,e90d1c98-55c4-47a8-a4f2-f857cf9ea02a
52fdbe02-eac2-41d4-a219-d2a17ba1f3eb,wilbur20210512_YDIE20AUQI.nwb,97cbb105-7a60-4f5d-8cc3-c622e4c76112
5a34ae36-f907-4e37-8cd5-10330c3b3697,wilbur20210512_NLGWNS9EYA.nwb,5d30cd78-a322-4d09-9d1d-8c95c21810b8
61744f96-5df5-4520-abe0-e903a5ccb8e6,wilbur20210512_SYBL05AX1C.nwb,466b2080-f857-4030-9803-a03956c7eaa6
74581524-b4df-4a88-98c9-f797bb3ac0d0,wilbur20210512_VDAF98R66Y.nwb,79e7177a-9360-4690-be77-8f115eda6c52
77055aa0-a603-465e-a399-0fff73cb549a,wilbur20210512_BIG3ISP1CB.nwb,b21c395a-37b7-42a0-86cb-ef3f4a6ec6c5
7c020084-fa8d-4d10-8cd0-eec51992008e,wilbur20210512_XT4AFCUGBA.nwb,278077db-ef6a-4804-94f9-fcca5be51d3d
7c11745d-b970-4d42-8f7d-fa62ce1cd908,wilbur20210512_FH6IGEIED8.nwb,ae02c2a9-fe62-4e8c-801b-a3ab0810842c
94946639-6b8a-4fce-bb0c-a1c4722522ec,wilbur20210512_D7HOOSCW9I.nwb,54c34a59-d463-4f24-821d-096240908eee


#### Downstream merge


In [21]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
from spyglass.spikesorting.analysis.v1.group import UnitSelectionParams
from spyglass.spikesorting.analysis.v1.group import SortedSpikesGroup
from spyglass.spikesorting.analysis.v1.unit_annotation import UnitAnnotation

In [10]:
merge_insert_key = (sgs.CurationV1()  & {"curation_id":2} & f'analysis_file_name LIKE "{prefix}\\_%"').fetch("KEY", as_dict = True)

SpikeSortingOutput.insert(merge_insert_key, part_name = "CurationV1")
SpikeSortingOutput()

merge_id,source
047f4da5-c1d4-ca9e-f923-cdb63c0d04e7,CurationV1
0670f62a-3df6-336c-4d38-a0a8761ca8fa,CurationV1
07500c97-ae8d-7578-d274-dd752ecbf155,CurationV1
0a29f32c-7036-e6bb-fd45-dc7ac35137fe,CurationV1
0b9cded7-0495-e4da-40b1-781b33a8620d,CurationV1
1a818a97-6f62-8d75-565f-7be81c6961c3,CurationV1
3b2a05fa-9642-5ef4-58d7-1f8e9bf3e217,CurationV1
4db00acd-003c-de9a-b640-fb78fb276985,CurationV1
652a4993-7ba0-6698-bbf1-feafd6d7fc4c,CurationV1
6dff3f6f-796e-2ca2-bef9-c077863b4997,CurationV1


In [79]:
SortedSpikesGroup()

nwb_file_name name of the NWB file,unit_filter_params_name,sorted_spikes_group_name
J1620210722_.nwb,default,epoch 2 mPFC
J1620210722_.nwb,default,epoch 2 OFC
J1620210722_.nwb,default,epoch 2 PFC
J1620210722_.nwb,default,epoch 4 mPFC
J1620210722_.nwb,default,epoch 4 OFC
J1620210722_.nwb,default,epoch 4 PFC
wilbur20210512_.nwb,default,left mPFC
wilbur20210512_.nwb,default,left OFC
wilbur20210512_.nwb,default,mPFC
wilbur20210512_.nwb,default,right mPFC


In [77]:
output_key = {**session_restrict, "interval_list_name": "wtrack_all", "curation_id": 2}
merge_ids = SpikeSortingOutput().get_restricted_merge_ids(output_key)
merge_keys = [{"merge_id": str(mid)} for mid in merge_ids]
SpikeSortingOutput().get_sort_group_info(merge_keys)


l_ofc_merge_ids =  (SpikeSortingOutput().get_sort_group_info(merge_keys) & {"electrode_group_name":34}).fetch("merge_id")
l_mpfc_merge_ids =  (SpikeSortingOutput().get_sort_group_info(merge_keys) & {"electrode_group_name":32}).fetch("merge_id")
r_mpfc_merge_ids =  (SpikeSortingOutput().get_sort_group_info(merge_keys) & {"electrode_group_name":33}).fetch("merge_id")


[16:40:24][WARNING] Spyglass: V0 requires artifact restrict. Ignoring "restrict_by_artifact" flag.


In [22]:
UnitSelectionParams.insert1({
    "unit_filter_params_name":"default",
    "include_labels": ['accept'],
    "exclude_labels" : ['reject', 'noise', 'mua', 'artifact'],
}, skip_duplicates = True)
UnitSelectionParams().insert_default()
UnitSelectionParams().fetch()

array([('all_units', list([]), list([])),
       ('default', list(['accept']), list(['reject', 'noise', 'mua', 'artifact'])),
       ('default_exclusion', list([]), list(['noise', 'mua'])),
       ('exclude_noise', list([]), list(['noise', 'mua']))],
      dtype=[('unit_filter_params_name', 'O'), ('include_labels', 'O'), ('exclude_labels', 'O')])

In [96]:
unit_filter_params_name = "default"
SortedSpikesGroup().create_group(
    group_name="mPFC",
    nwb_file_name=nwb_copy_file_name,
    keys = [
        {"spikesorting_merge_id": merge_id}
        for merge_id in np.append(r_mpfc_merge_ids, l_mpfc_merge_ids)
    ],
    unit_filter_params_name=unit_filter_params_name,
)

rmpfc_spikes_group_key = {**session_restrict, "sorted_spikes_group_name":"right mPFC"}
lmpfc_spikes_group_key = {**session_restrict, "sorted_spikes_group_name":"left mPFC"}
lofc_spikes_group_key = {**session_restrict, "sorted_spikes_group_name": "left OFC"}
#                     "nwb_file_name": nwb_copy_file_name}
SortedSpikesGroup() & session_restrict

[2025-12-26 18:16:53,660][WARNING]: MySQL server has gone away. Reconnecting to the server.


nwb_file_name name of the NWB file,unit_filter_params_name,sorted_spikes_group_name
wilbur20210512_.nwb,all_units,mPFC all
wilbur20210512_.nwb,default,left mPFC
wilbur20210512_.nwb,default,left OFC
wilbur20210512_.nwb,default,mPFC
wilbur20210512_.nwb,default,right mPFC


In [67]:
r_mpfc_spike_times, r_mpfc_unit_ids = SortedSpikesGroup().fetch_spike_data(rmpfc_spikes_group_key, return_unit_ids=True)
l_mpfc_spike_times, l_mpfc_unit_ids = SortedSpikesGroup().fetch_spike_data(lmpfc_spikes_group_key, return_unit_ids=True)
l_ofc_spike_times, l_ofc_unit_ids = SortedSpikesGroup().fetch_spike_data(lofc_spikes_group_key, return_unit_ids=True)



In [87]:
print("# l mPFC units: ", len(l_mpfc_unit_ids))
print("# r mPFC units: ", len(r_mpfc_unit_ids))
print("# r OFC units: ", len(l_ofc_unit_ids))

# l mPFC units:  205
# r mPFC units:  58
# r OFC units:  7
